In [22]:
%pip install torch
%pip install numpy
%pip install pandas
%pip install openpyxl
%pip install onnx
%pip install onnxscript

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Read raw data

In [23]:
import json
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [24]:
rawData = pd.read_excel('D:\external-projects\cautious-parakeet\Controller\Control\data.xlsx', header=None, names=['Id', 'BotInput', 'BotCommand'])


<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\tiyani.mabunda\AppData\Local\Temp\ipykernel_18952\181927427.py:1: SyntaxWarning: invalid escape sequence '\e'
  rawData = pd.read_excel('D:\external-projects\cautious-parakeet\Controller\Control\data.xlsx', header=None, names=['Id', 'BotInput', 'BotCommand'])


Data visualization 

Clean data

In [25]:


def preprocess_botcommand(data):
 

  data_list = []
  for item in data:
    # Extract numerical features
    parsed_data = json.loads(item)

    move_x = parsed_data['Move']['X']
    move_y = parsed_data['Move']['Y']
    rotate = parsed_data['Rotate']
    shoot =  parsed_data['Shoot']
    # Create feature vector
    feature_vector = [move_x, move_y, rotate,shoot] 
    data_list.append(feature_vector)

  df = pd.DataFrame(data_list, columns=['Move_X', 'Move_Y', 'Rotate','Shoot']) 
  return df
def preprocess_botinput(input) :

  
  parsed_data = json.loads(input)
  player_x = parsed_data['Player']['Position']['X']
  player_y = parsed_data['Player']['Position']['Y']
  player_velocity = parsed_data['Player']['Velocity']['SqrMagnitude']
  player_rotation = parsed_data['Player']['Health']
  player_health = parsed_data['Player']['Health']
  player_score = parsed_data['Player']['Score']
  feature_vector = [player_x, player_y, player_velocity,player_rotation,player_health,player_score]  
  

  #df = pd.DataFrame([feature_vector], columns=['Player_X', 'Player_Y', 'Player_Velocity','Player_Rotation','Player_Health','Player_Score'])
  return torch.tensor(feature_vector, dtype=torch.float32)

In [26]:
labels = preprocess_botcommand(rawData["BotCommand"])
#features = preprocess_botinput(rawData["BotInput"]).values


Model definition

In [27]:


def bot_command_builder(move,rotate,shoot):
  
  command = {
    "Move": 
    {
      "X": move[0].item(),
      "Y": move[1].item(),
      "SqrMagnitude": 0.0
    },
    "Rotate": rotate.item(),
    "Shoot": shoot[0].item() > 0.5
  }
  
  return command

class BotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size_move, output_size_rotate):
        super(BotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2_move = nn.Linear(hidden_size, output_size_move)
        self.fc2_rotate = nn.Linear(hidden_size, output_size_rotate)
        self.fc2_shoot = nn.Linear(hidden_size, 1)  # For classification

    def forward(self, x):
        
        x = torch.relu(self.fc1(x))
        move = self.fc2_move(x)
        rotate = self.fc2_rotate(x)
        shoot = torch.sigmoid(self.fc2_shoot(x))  # For classification
        #command = bot_command_builder(move, rotate, shoot)
        return move , rotate , shoot

Training 

In [28]:
input_size = 6  # Replace with actual input size
hidden_size = 64
output_size_move = 2
output_size_rotate = 1
num_epochs = 20
learning_rate = 0.001

In [29]:
def to_tensor(data , dtype = torch.float32):
    return torch.tensor(data,dtype=dtype)
    
def train(model, X,x_move,y_move, rotate, shoot, num_epochs, learning_rate):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    loss_fn_shoot = nn.BCELoss()

    for epoch in range(num_epochs):
        for i in range(len(X)):
            optimizer.zero_grad()
            move_pred , rotate_pred , shoot_pred = model(X[i]) 
            loss_move= loss_fn(move_pred, torch.tensor([x_move[i] , y_move[i]],dtype=torch.float32))
            loss_rotate = loss_fn(rotate_pred ,rotate[i])
            loss_shoot = loss_fn_shoot( shoot_pred[0] , shoot[i] )  # Convert to float for BCELoss
            loss =   loss_shoot + loss_rotate +loss_move
            loss.backward()
            optimizer.step()

In [30]:
model_data = rawData["BotInput"].apply(preprocess_botinput)

x_move = torch.tensor(labels['Move_X'], dtype=torch.float32)
y_move = torch.tensor(labels['Move_Y'], dtype=torch.float32)

rotate = torch.tensor(labels['Rotate'], dtype=torch.float32)
shoot = torch.tensor(labels['Shoot'], dtype=torch.float32)
model = BotModel(input_size, hidden_size, output_size_move, output_size_rotate)

train(model, model_data.values, x_move,y_move, rotate, shoot, num_epochs, learning_rate)


d:\apps\miniconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [31]:
onnx_program = torch.onnx.dynamo_export(model, model_data.values[0])
 
onnx_program.save("bot.onnx")

d:\apps\miniconda3\Lib\site-packages\torch\onnx\_internal\exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


Applied 4 of general pattern rewrite rules.


Evaluations